# PixelSounds. Visualización de audio en vídeo con Python
### Introducción

En este caso, y siguiendo en la línea de los anteriores conversores propuestos en el proyecto PixelSounds, vamos a tratar de convertir audio a vídeo. Sin embargo, no va a ser cualquier tipo de vídeo. Vamos a tratar de generar un vídeo que nos permita visualizar en una primera vista algunas características del audio que hay detrás. Para ello vamos a extraer algunos descriptores del audio (los cuales especificaremos más adelante) y vamos a transformarlos en algo visual. De esta forma, iremos generando fotogramas y mediante la libreria `ffmpeg` los uniremos para crear un video. En este cuaderno vamos a ir desgranando una por una las transformaciones que haremos para visualizar los distintos descriptores. Después uniremos todas las funciones para generar un fotograma. Tras aprender a generar un fotograma, podremos generar muchos para crear el vídeo.

### Librerías necesarias

In [5]:
modules = {
    "numpy": "numpy",
    "matplotlib": "matplotlib",
    "IPython": "ipywidgets",
    "scipy": "scipy",
    "librosa": "librosa",
    "ipywidgets": "ipywidgets",
    "bqplot": "bqplot",
    "pyaudio": "pyaudio",
    "ffmpeg" : "ffmpeg-python"
}

for mod, pip_name in modules.items():
    try:
        __import__(mod)
    except ImportError:
        if pip_name:
            print(f"🔧 Instalando {pip_name}...")
            %pip install {pip_name}
        else:
            print(f"⚠️ Módulo {mod} no se puede instalar automáticamente (builtin o personalizado)")

🔧 Instalando ffmpeg-python...
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
import vozyaudio as vz
import numpy as np
import matplotlib.pyplot as plt
import subprocess
import os
# Instalar ffmpeg-python
import ffmpeg
import shutil
import colorsys
from vozyaudio import lee_audio, envolvente, track_pitch, espectro
from scipy.signal import resample, correlate, find_peaks
from IPython.display import Video

### Los primeros pasos
Antes de comenzar a hacer nada, vamos a definir algunos parámetros básicos que vamos a utilizar más adelante. La configuración del codificador consta de los siguientes parámetros.

In [7]:
# Configuración
AUDIO_PATH = 'audios/music.wav' # Ruta del audio que vamos a usar
FPS = 25 # Número de fotogramas por segundo del vídeo resultante
FRAME_FOLDER = 'fotogramas' # Ruta de la carpeta donde iremos guardando los fotogramas
N_BARRAS = 60  # Número de barras del espectro

# Borrar las carpetas de salida si existen
for d in ("fotogramas", "pruebas"):
    if os.path.exists(d):
        print(f"Eliminando carpeta existente: {d}/")
        shutil.rmtree(d)
    else:
        print(f"No existe: {d}/ — nada que borrar.")

# Crear carpetas de salida
frames_dir = f"fotogramas"
export_dir = f"pruebas"

os.makedirs(frames_dir, exist_ok=True)
print(f"Carpeta {frames_dir} creada.")
os.makedirs(export_dir, exist_ok=True)
print(f"Carpeta {export_dir} creada.")

No existe: fotogramas/ — nada que borrar.
No existe: pruebas/ — nada que borrar.
Carpeta fotogramas creada.
Carpeta pruebas creada.


Después de esto, cargaremos el audio y extraeremos su duración, número de muestras, y muestras por fotograma. Además, obtendremos el número de frames que ha de tener el vídeo resultante.

In [8]:
# Cargar audio
fs, x = lee_audio(AUDIO_PATH) # Lee el audio
x = x.astype(np.float32) # Transforma el tipo de dato
dur = len(x) / fs # Duración del audio
n_frames = int(FPS * dur) # Número de frames del vídeo
samples_per_frame = int(fs / FPS) # Número de muestras 

Tras hacer todo esto ya podemos empezar a generar las primeras componentes del fotograma.

### Círculo que se mueve con el pitch

Lo primero que vamos a agregar al fotograma es un **circulo que se mueve de arriba a abajo con el pitch y cambia de tamaño con la envolvente**. Para este paso, usaremos las funciones `envolvente` y `track_pitch` del módulo `vozyaudio` para extraer la envolvente y estimar el pitch del audio. Después normalizaremos ambos descriptores y los redimensionaremos para que se ajusten al número de frames finales del video. Todo esto lo haremos dentro de la función `obtener_descriptores` 

In [9]:
def normalizar(v):
    """ Normaliza un vector al rango [0, 1].
    Entrada:
        v (numpy.ndarray): Vector de valores (por ejemplo, envolvente, pitch, espectro, etc.)
    Salida:
        v_norm (numpy.ndarray): Vector normalizado en el rango [0, 1]
    """
    return (v - np.min(v)) / (np.max(v) - np.min(v) + 1e-9)

def obtener_descriptores(x,fs):
    """ Extrae distintos descriptores de la señal de audio.
    Entrada:
        x (numpy.ndarray): Vector de valores de la señal
        fs (int): Frecuencia de muestreo de la señal
    Salida:
        pitch_frame (numpy.ndarray): Vector con los valores de la estimación del pitch normalizados y redimensionados al número de frames
        env_frame (numpy.ndarray): Vector con los valores de la envolvente normalizados y redimensionados al número de frames 
    """
    # Extraemos los descriptores de envolvente y estimación de pitch
    env = envolvente(x, fs=fs) # Extraer envolvente
    pitch = track_pitch(x, fs) # Estimar pitch
    pitch = np.nan_to_num(pitch)  # Reemplaza NaNs por 0
    
    env = normalizar(env) # Normalizar ambos arrays
    pitch = normalizar(pitch)
    
    # Redimensionar descriptores al número de frames
    env_frame = np.interp(np.linspace(0, len(env), n_frames), np.arange(len(env)), env)
    pitch_frame = np.interp(np.linspace(0, len(pitch), n_frames), np.arange(len(pitch)), pitch)

    return pitch_frame, env_frame


Con estas variables ya tenemos todo lo necesario para generar el primer componente que variará durante el vídeo. Lo siguiente que debemos hacer es definir una función que dibujo el circulo en pantalla variando segúne estos valores. A esta función la llamaremos `dibujar_particula`.

In [10]:
def dibujar_particula(pitch, env):
    """ Dibuja una partícula que se mueve según el pitch y cambia de tamaño según la envolvente del audio
    Entrada:
        pitch (float) : Valor de estimación del pitch en un fotograma determinado
        env (float) : Valor de la envolvente del audio en un fotograma determinado
    Salida:
        None: La función solo dibuja en la figura y no devuelve nada
    """
    y_pos = pitch
    size = 100 + env * 300
    color = (1.0, env, pitch)
    plt.scatter(0.5, y_pos, s=size, c=[color], alpha=0.8)

Para visualizar que el resultado es el que esperamos tenemos que definir una función básica de generación de frames. Esta función la iremos ampliando a medida que vayamos añadiendo más componentes al fotograma. Cuando tengamos todos los fotogramas generados los uniremos mediante la función `crear_video`  para ver el resultado. Esta función hace uso de `subprocess` y de `ffmpeg` para crear el video de manera rápida simplemente pasándole el archivo `generarVideo.bat` que se encargará de ejecutar los comandos ffmpeg. La usaremos mucho a lo largo del cuaderno. De momento, probemos a generar los fotogramas con solo esta componente y ver el video resultado.

In [11]:
def generar_frames(x,fs, FRAME_FOLDER):
    """ Genera los frames para el vídeo y los guarda en la carpeta FRAME_FOLDER
    Entrada:
        x (numpy.ndarray): Vector de valores de la señal
        fs (int): Frecuencia de muestreo de la señal
        FRAME_FOLDER (string): Ruta de la carpeta destino
    Salida:
        None: No devuelve nada, solo genera los fotogramas      
    """
    pitch_frame, env_frame = obtener_descriptores(x,fs)
    
    print("Generando frames...")
    for i in range(n_frames):
        porcentaje = (i / (n_frames-1)) * 100
        print(f"\rCompletado {porcentaje:.2f} %", end="", flush=True)

        fig, ax = plt.subplots(figsize=(6, 4))

        # Visual: Círculo que sube/baja con pitch y cambia tamaño con envolvente
        dibujar_particula(pitch_frame[i], env_frame[i])

        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.axis('off')

        plt.tight_layout()
        plt.savefig(f"{FRAME_FOLDER}/frame_{i:04d}.png")
        plt.close(fig)
    
generar_frames(x,fs,FRAME_FOLDER)
out = "pruebas/pruebaPitch.mp4"

Generando frames...
Completado 100.00 %

In [18]:
def crear_video(audio_path,out):
    try:
        if os.path.exists(out):
            os.remove(out)
        subprocess.run(
        ["cmd", "/c", "generarVideo.bat", audio_path, out],
        check=True
        )
    except subprocess.CalledProcessError as e:
        print("Error al ejecutar generarVideo.bat:", e)
    finally:
        # shutil.rmtree('fotogramas/')
        print('\nProcesado terminado.')
        

In [19]:
crear_video(AUDIO_PATH,out)

Error al ejecutar generarVideo.bat: Command '['cmd', '/c', 'generarVideo.bat', 'audios/music.wav', 'pruebas/pruebaPitch.mp4']' returned non-zero exit status 1.

Procesado terminado.


A continuación visualizaremos el resultado en la siguiente celda. Podemos ver como la posición vertical varia con el pitch y la intensidad del halo varia con la envolvente.

In [17]:
Video("pruebas/pruebaPitch.mp4")

ValueError: To embed videos, you must pass embed=True (this may make your notebook files huge)
Consider passing Video(url='...')

### Barras espectrales

Después de hacer que la estimación del pitch sea visible en el video vamos a añadir algún componente que nos muestre de alguna forma la **energía de cada banda de frecuencia en la señal**. Para ello vamos a usar barras espectrales que aumenten y disminuyan en función de la energía espectral de la señal. Para ello haremos uso de la función `espectro`. Lo que vamos a hacer es dividir la señal en trozos. De cada trozo extraeremos su información espectral y las adapataremos al componente visual de la barras. Todo esto lo introduciremos dentro de la función `generar_frames`.

La primera función que vamos a desarrollar en este bloque es la de `dibujar_barras`. Se encargará de plotear las barras en los fotogramas.

In [23]:
def dibujar_barras(X_resampled, N_BARRAS):
    """ Dibuja barras verticales que representan la energía en diferentes bandas de frecuencia.
    Entrada:
        X_resampled (numpy.ndarray): Vector con las amplitudes espectrales reescaladas a N_BARRAS bandas
        N_BARRAS (int): Número total de barras a dibujar
    Salida:
        None: Las barras se dibujan directamente sobre la figura
    """
    bar_width = 1 / N_BARRAS
    for j in range(N_BARRAS):
        height = X_resampled[j]
        color = (0.1, 0.8 * height, 1.0)
        plt.bar(j * bar_width, height, width=bar_width*0.8, color=color, align='edge')

Tras esto, vamos a añadir a `generar_frames` la parte de dividir la señal en trozos, extraer su espectro y dibujar las barras espectrales.

In [24]:
def generar_frames(x,fs, FRAME_FOLDER):
    """ Genera los frames para el vídeo y los guarda en la carpeta FRAME_FOLDER
    Entrada:
        x (numpy.ndarray): Vector de valores de la señal
        fs (int): Frecuencia de muestreo de la señal
        FRAME_FOLDER (string): Ruta de la carpeta destino
    Salida:
        None: No devuelve nada, solo genera los fotogramas      
    """
    pitch_frame, env_frame = obtener_descriptores(x,fs)
    
    print("Generando frames...")
    for i in range(n_frames):
        porcentaje = (i / (n_frames-1)) * 100
        print(f"\rCompletado {porcentaje:.2f} %", end="", flush=True)

        fig, ax = plt.subplots(figsize=(6, 4))
        
        # ------NUEVO--------
        
        #  Obtener trozo de señal actual
        start = i * samples_per_frame
        end = min(len(x), start + samples_per_frame)
        x_frame = x[start:end]

        # Espectro (resample a N barras)
        X, fa = espectro(x_frame, modo=1, fs=fs)
        X_resampled = resample(X, N_BARRAS)
        X_resampled = normalizar(X_resampled)
        
        # ------NUEVO--------

        # Visual: Círculo que sube/baja con pitch y cambia tamaño con envolvente
        dibujar_particula(pitch_frame[i], env_frame[i])
        
        dibujar_barras(X_resampled, N_BARRAS)

        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.axis('off')

        plt.tight_layout()
        plt.savefig(f"{FRAME_FOLDER}/frame_{i:04d}.png")
        plt.close(fig)
    
generar_frames(x,fs,FRAME_FOLDER)

out = "pruebas/pruebaBarras.mp4"
crear_video(AUDIO_PATH,out)

Generando frames...
Completado 100.00 %
Procesado terminado.


En la siguiente celda podemos observar como las barras verticales varian con la energía de la señal.

In [25]:
Video("pruebas/pruebaBarras.mp4")

### Autocorrelación y patrones rítmicos

La autocorrelación mide cómo una señal se parece a sí misma desplazada en el tiempo. En música, eso se traduce en:

* Picos periódicos en la autocorrelación = ritmo repetitivo o beats.

* Puede ayudarte a detectar tempo, pulsos o patrones repetitivos como los que tienen bases de batería, loops, etc.

Por esto sabemos que la autocorrelación es una herramienta realmente potente para analizar el contenido rítmico de un audio. Lo primero que vamos a hacer es definir una función que nos ayude a calcular la autocorrelación de una señal de audio. Para ello nos ayudaremos de la función `correlate` del módulo `scipy.signal`.

In [26]:
def autocorrelacion(x_frame):
    """ Calcula la autocorrelación normalizada de una ventana de señal de audio.
    Entrada:
        x_frame (numpy.ndarray): Fragmento de señal de audio (ventana temporal)
    Salida:
        corr_norm (numpy.ndarray): Autocorrelación normalizada desde el retardo cero hacia adelante
    """
    x_frame = x_frame - np.mean(x_frame)
    corr = correlate(x_frame, x_frame, mode='full')
    mid = len(corr) // 2
    return corr[mid:] / np.max(np.abs(corr) + 1e-9)

Una vez hecho esto podemos crear una función `detectar_ritmo` que estime el ritmo de un fragmento de audio usando la correlación, y con este ritmo podemos crear un **círculo en el centro del frame que lata con intensidad variante según el ritmo**. Para ello utilizaremos `sin(2π * t / periodo)`. Con `detectar_ritmo` tenemos todo lo necesario para crear la función `dibujar_circulo_ritmico`.

In [27]:
def detectar_ritmo(x_frame, fs, fmin=1.5, fmax=8):
    """ Estima el periodo rítmico de un fragmento de audio mediante autocorrelación.
    Entrada:
        x_frame (numpy.ndarray): Fragmento de señal de audio (ventana temporal)
        fs (int): Frecuencia de muestreo del audio
        fmin (float): Frecuencia mínima esperada del ritmo (en Hz)
        fmax (float): Frecuencia máxima esperada del ritmo (en Hz)
    Salida:
        periodo_seg (float): Periodo estimado del ritmo en segundos
        corr (numpy.ndarray): Autocorrelación normalizada del fragmento de audio
    """
    corr = autocorrelacion(x_frame)
    min_lag = int(fs / fmax)
    max_lag = int(fs / fmin)
    if max_lag >= len(corr): max_lag = len(corr) - 1
    if min_lag >= max_lag: return 0.5, corr  # Valor por defecto
    pico = np.argmax(corr[min_lag:max_lag]) + min_lag
    periodo_seg = pico / fs
    return periodo_seg, corr

def dibujar_circulo_ritmico(t_actual, periodo):
    """ Dibuja un círculo que pulsa rítmicamente en el centro del frame según un periodo dado.
    Entrada:
        t_actual (float): Tiempo actual del video en segundos
        periodo (float): Periodo rítmico estimado en segundos
    Salida:
        None: El círculo se dibuja directamente sobre la figura
    """
    ritmo_osc = 0.5 * (1 + np.sin(2 * np.pi * t_actual / periodo))
    color = (ritmo_osc, 0.2, 1 - ritmo_osc)
    size = 300 * ritmo_osc + 20
    plt.scatter(0.5, 0.5, s=size, c=[color], alpha=0.3)

Ahora añadiremos toda esta información al bucle de generación del fotograma. Aprovecharemos la división a trozos de la señal que hemos implementado antes.

In [28]:
def generar_frames(x,fs, FRAME_FOLDER):
    """ Genera los frames para el vídeo y los guarda en la carpeta FRAME_FOLDER
    Entrada:
        x (numpy.ndarray): Vector de valores de la señal
        fs (int): Frecuencia de muestreo de la señal
        FRAME_FOLDER (string): Ruta de la carpeta destino
    Salida:
        None: No devuelve nada, solo genera los fotogramas      
    """
    pitch_frame, env_frame = obtener_descriptores(x,fs)
    
    print("Generando frames...")
    for i in range(n_frames):
        porcentaje = (i / (n_frames-1)) * 100
        print(f"\rCompletado {porcentaje:.2f} %", end="", flush=True)

        fig, ax = plt.subplots(figsize=(6, 4))
        
        #  Obtener trozo de señal actual
        start = i * samples_per_frame
        end = min(len(x), start + samples_per_frame)
        x_frame = x[start:end]

        # Espectro (resample a N barras)
        X, fa = espectro(x_frame, modo=1, fs=fs)
        X_resampled = resample(X, N_BARRAS)
        X_resampled = normalizar(X_resampled)
        
        # ------NUEVO------
        
        # Detección rítmica simple
        periodo, corr = detectar_ritmo(x_frame, fs)
        
        # Calcula un pulso visual que oscila con el ritmo detectado
        t_actual = i / FPS
        
        dibujar_circulo_ritmico(t_actual,periodo)
        
         # ------NUEVO------

        # Visual: Círculo que sube/baja con pitch y cambia tamaño con envolvente
        dibujar_particula(pitch_frame[i], env_frame[i])
        
        dibujar_barras(X_resampled, N_BARRAS)

        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.axis('off')

        plt.tight_layout()
        plt.savefig(f"{FRAME_FOLDER}/frame_{i:04d}.png")
        plt.close(fig)
    
generar_frames(x,fs,FRAME_FOLDER)

out = "pruebas/pruebaRitmo.mp4"
crear_video(AUDIO_PATH,out)

Generando frames...
Completado 100.00 %
Procesado terminado.


Ahora visualizaremos el resultado.

In [29]:
Video("pruebas/pruebaRitmo.mp4")

Podemos observar como el circulo late de forma constante al ritmo constante delimitado por los beats de fondo del audio original. Sin embargo, este diseño que acabamos de crear tiene **una debilidad importante**, y es que no es capaz de distinguir qué ritmo del audio tiene contenido armónico relevante para nosotros. Debido a que estamos aplicando la autocorrelación directamente sobre la onda real podemos estar obteniendo partes del ritmo que son poco relevantes en armónicamente en la canción.

Por ello, para hacer un aislamiento de estas partes poco relevantes y quedarnos con lo que nos interesa de verdad (y en consecuencia hacer que el circulo lata todavía más acorde con la canción), **vamos a aplicar la autocorrelación sobre la envolvente**. De esta forma vamos a obtener los **golpes rítmicos reales** de la señal. Por cada golpe rítmico vamos a generar un flash en el centro del fotograma.

In [30]:
def beat_frames(x, fs):
    """ Detecta los beats del audio a partir de la envolvente y devuelve sus ubicaciones en frames.
    Entrada:
        x (numpy.ndarray): Señal de audio completa
        fs (int): Frecuencia de muestreo del audio
    Salida:
        beat_frames (numpy.ndarray): Índices de frame donde se detectan beats en la señal
    """
    # Autocorrelación sobre la envolvente
    env_smooth = envolvente(x, fs=fs, tr=0.1)  # más estable
    corr_env = autocorrelacion(env_smooth)

    # Estimar el tempo global
    min_lag = int(fs / 5)    # máx 5 Hz = 300 BPM
    max_lag = int(fs / 1.5)  # mín 1.5 Hz = 90 BPM
    lag_beat = np.argmax(corr_env[min_lag:max_lag]) + min_lag
    periodo_muestras = lag_beat

    # Encontrar los picos en la envolvente
    peaks, _ = find_peaks(env_smooth, distance=periodo_muestras * 0.8)

    # Convertir los picos (en muestras) a tiempos (en segundos) y luego a frames
    beat_times = peaks / fs
    beat_frames = (beat_times * FPS).astype(int)
    return beat_frames

def es_beat(i, beat_frames, tolerancia=2):
    """ Determina si un frame está dentro de los limites de un beat detectado.
    Entrada:
        frame_index (int): Índice del frame actual en el video
        beat_frames (list of int): Lista de frames donde se detectaron beats
        tolerancia (int): Número de frames de margen alrededor de cada beat
    Salida:
        es_beat (bool): True si el frame está cerca de un beat, False en caso contrario
    """
    return any(abs(i - bf) <= tolerancia for bf in beat_frames)


def dibujar_flash(i, beats):
    """ Dibuja un flash visual en el centro del frame, usado para resaltar beats detectados.
    Entrada:
        beats (numpy.ndarray): Array con los índices de los fotogramas donde hay un beat
    Salida:
        None: El flash se dibuja directamente sobre la figura
    """
    # Halo pulsante animado en el centro tras beat
    for bf in beats:
        frames_from_beat = i - bf
        if 0 <= frames_from_beat <= 4:  # duración 4 frames
            grow = 1 - frames_from_beat / 4
            size = 2000 * grow
            alpha = 0.8 * grow
            plt.scatter(0.5, 0.5, s=size, c='magenta', alpha=alpha, edgecolors='none')


Ahora vamos a ampliar la función `generar_frames`.

In [31]:
def generar_frames(x,fs, FRAME_FOLDER):
    """ Genera los frames para el vídeo y los guarda en la carpeta FRAME_FOLDER
    Entrada:
        x (numpy.ndarray): Vector de valores de la señal
        fs (int): Frecuencia de muestreo de la señal
        FRAME_FOLDER (string): Ruta de la carpeta destino
    Salida:
        None: No devuelve nada, solo genera los fotogramas      
    """
    pitch_frame, env_frame = obtener_descriptores(x,fs)
    
    print("Generando frames...")
    for i in range(n_frames):
        porcentaje = (i / (n_frames-1)) * 100
        print(f"\rCompletado {porcentaje:.2f} %", end="", flush=True)

        fig, ax = plt.subplots(figsize=(6, 4))
        
        
        #------NUEVO------
        
        beats = beat_frames(x,fs)
        
        # Flash más visible en beat
        if es_beat(i,beats,2):  # mayor tolerancia
            plt.scatter(0.5, 0.5, s=1500, c='cyan', alpha=0.9, edgecolors='none', marker='o')
        
        dibujar_flash(i,beats)
            
        #------NUEVO------
            
        #  Obtener trozo de señal actual
        start = i * samples_per_frame
        end = min(len(x), start + samples_per_frame)
        x_frame = x[start:end]

        # Espectro (resample a N barras)
        X, fa = espectro(x_frame, modo=1, fs=fs)
        X_resampled = resample(X, N_BARRAS)
        X_resampled = normalizar(X_resampled)
        
        # Detección rítmica simple
        periodo, corr = detectar_ritmo(x_frame, fs)
        
        # Calcula un pulso visual que oscila con el ritmo detectado
        t_actual = i / FPS
        
        dibujar_circulo_ritmico(t_actual,periodo)

        # Visual: Círculo que sube/baja con pitch y cambia tamaño con envolvente
        dibujar_particula(pitch_frame[i], env_frame[i])
        
        dibujar_barras(X_resampled, N_BARRAS)

        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.axis('off')

        plt.tight_layout()
        plt.savefig(f"{FRAME_FOLDER}/frame_{i:04d}.png")
        plt.close(fig)
    
generar_frames(x,fs,FRAME_FOLDER)

out = "pruebas/pruebaRitmo2.mp4"
crear_video(AUDIO_PATH,out)

Generando frames...
Completado 100.00 %
Procesado terminado.


In [32]:
Video("pruebas/pruebaRitmo2.mp4")

Como vemos, cada vez tarda más en generar los fotogramos puesto que estamso todo el rato añadiendo nueva a información a la funciçon `generar_frames`. Con este último componente en el vídeo ya podemos dar por finalizado todo el proceso de este conversor de audio a vídeo. Sin embargo, nos gustaría añadir un último apartado con algo que nos parece curioso pero que no queremos incluir en la versión final ya que el resultado es bastante epileptico y la idea está sacada de Internet. Lo dejamos como celda opcional a ejecutar. Si no quieres hacerlo, salta directamente al apartado del resultado final.

### Opcional: variación de color de fondo con centroide espectral

Para darle un toque final al programa estábamos pensando en jugar con la tonalidad del audio. Queriamos encontrar algo que variase según los cambios de tonalidad (cambio de grave a agudo etc). Se nos ocurrió que podríamos variar el color de fondo y, buscando por Internet, encontramos algo que podemos obtener a partir del espectro de la señal: **el centroide espectral**. El centroide espectral indica la "brillantez" del sonido, es decir, qué tan concentrada está la energía en frecuencias altas.

* Un centroide alto → sonidos agudos o brillantes → fondo más claro o cálido.

* Un centroide bajo → sonidos graves o apagados → fondo más oscuro o frío.

In [ ]:
import colorsys
from vozyaudio import espectro
import numpy as np

import colorsys
from vozyaudio import espectro
import numpy as np

def color_fondo_por_centroide(X, fa, x_frame, fs):
    """ Genera un color RGB para el fondo en función del centroide espectral del frame.
    Entrada:
        X (np.ndarray): Módulo del espectro del frame de audio (magnitudes)
        fa (np.ndarray): Vector de frecuencias correspondientes al espectro
        x_frame (np.ndarray): Fragmento de señal de audio correspondiente al frame
        fs (int): Frecuencia de muestreo del audio
    Salida:
        fondo_color (tuple): Color RGB normalizado (0-1) para usar como fondo del frame
    """
    # Evitar errores si X está vacío
    if np.sum(X) == 0 or len(fa) != len(X):
        return (0, 0, 0.1)  # fondo oscuro por defecto

    # Centroide espectral
    centroide = np.sum(fa * X) / (np.sum(X) + 1e-9)

    # Normalizar centroide al rango 0–1 basado en un rango realista (0–4000 Hz)
    centroide_norm = np.clip(centroide / 4000, 0, 1)

    # Usar centroide como matiz, pero también afectar brillo
    h = centroide_norm                   # matiz (rojo ↔ azul)
    s = 0.9                              # saturación constante
    v = 0.3 + 0.7 * centroide_norm       # más agudo → más brillante

    fondo_color = colorsys.hsv_to_rgb(h, s, v)
    return fondo_color

Con esta función, la función `generar_frames` quedaría así.

In [ ]:
def generar_frames(x,fs, FRAME_FOLDER):
    """ Genera los frames para el vídeo y los guarda en la carpeta FRAME_FOLDER
    Entrada:
        x (numpy.ndarray): Vector de valores de la señal
        fs (int): Frecuencia de muestreo de la señal
        FRAME_FOLDER (string): Ruta de la carpeta destino
    Salida:
        None: No devuelve nada, solo genera los fotogramas      
    """
    pitch_frame, env_frame = obtener_descriptores(x,fs)
    
    print("Generando frames...")
    for i in range(n_frames):
        porcentaje = (i / (n_frames-1)) * 100
        print(f"\rCompletado {porcentaje:.2f} %", end="", flush=True)

        fig, ax = plt.subplots(figsize=(6, 4))
        
        beats = beat_frames(x,fs)
        
        # Flash más visible en beat
        if es_beat(i,beats,2):  # mayor tolerancia
            plt.scatter(0.5, 0.5, s=1500, c='cyan', alpha=0.9, edgecolors='none', marker='o')
        
        dibujar_flash(i,beats)
            
        #  Obtener trozo de señal actual
        start = i * samples_per_frame
        end = min(len(x), start + samples_per_frame)
        x_frame = x[start:end]

        # Espectro (resample a N barras)
        X, fa = espectro(x_frame, modo=1, fs=fs)
        X_resampled = resample(X, N_BARRAS)
        X_resampled = normalizar(X_resampled)
        
        #------NUEVO------
        
        fondo_color =  color_fondo_por_centroide(X,fa,x_frame,fs)
        fig.set_facecolor(fondo_color)
        
        #------NUEVO------
        
        # Detección rítmica simple
        periodo, corr = detectar_ritmo(x_frame, fs)
        
        # Calcula un pulso visual que oscila con el ritmo detectado
        t_actual = i / FPS
        
        dibujar_circulo_ritmico(t_actual,periodo)

        # Visual: Círculo que sube/baja con pitch y cambia tamaño con envolvente
        dibujar_particula(pitch_frame[i], env_frame[i])
        
        dibujar_barras(X_resampled, N_BARRAS)

        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.axis('off')

        plt.tight_layout()
        plt.savefig(f"{FRAME_FOLDER}/frame_{i:04d}.png")
        plt.close(fig)
    
generar_frames(x,fs,FRAME_FOLDER)

out = "pruebas/pruebaColor.mp4"
crear_video(AUDIO_PATH,out)

In [ ]:
Video("pruebas/pruebaColor.mp4")

### Resultado final

Para finalizar, vamos a dejar una celda con la función final de `generar_frames` y las rutas para poder cambiar fácilmente los audios y poder testear audios distintos al de ejemplo. Recomendamos testear con audios de duración no superior a 15 segundos.

In [33]:
# Configuración
AUDIO_PATH = 'audios/music.wav' # Ruta del audio que vamos a usar
FPS = 25 # Número de fotogramas por segundo del vídeo resultante
FRAME_FOLDER = 'fotogramas' # Ruta de la carpeta donde iremos guardando los fotogramas
N_BARRAS = 60  # Número de barras del espectro

# Cargar audio
fs, x = lee_audio(AUDIO_PATH) # Lee el audio
x = x.astype(np.float32) # Transforma el tipo de dato
dur = len(x) / fs # Duración del audio
n_frames = int(FPS * dur) # Número de frames del vídeo
samples_per_frame = int(fs / FPS) # Número de muestras 

def generar_frames(x,fs, FRAME_FOLDER):
    """ Genera los frames para el vídeo y los guarda en la carpeta FRAME_FOLDER
    Entrada:
        x (numpy.ndarray): Vector de valores de la señal
        fs (int): Frecuencia de muestreo de la señal
        FRAME_FOLDER (string): Ruta de la carpeta destino
    Salida:
        None: No devuelve nada, solo genera los fotogramas      
    """
    pitch_frame, env_frame = obtener_descriptores(x,fs)
    
    print("Generando frames...")
    for i in range(n_frames):
        porcentaje = (i / (n_frames-1)) * 100
        print(f"\rCompletado {porcentaje:.2f} %", end="", flush=True)

        fig, ax = plt.subplots(figsize=(6, 4))
        
        beats = beat_frames(x,fs)
        
        # Flash más visible en beat
        if es_beat(i,beats,2):  # mayor tolerancia
            plt.scatter(0.5, 0.5, s=1500, c='cyan', alpha=0.9, edgecolors='none', marker='o')
        
        dibujar_flash(i,beats)
            
        #  Obtener trozo de señal actual
        start = i * samples_per_frame
        end = min(len(x), start + samples_per_frame)
        x_frame = x[start:end]

        # Espectro (resample a N barras)
        X, fa = espectro(x_frame, modo=1, fs=fs)
        X_resampled = resample(X, N_BARRAS)
        X_resampled = normalizar(X_resampled)
        
        # Detección rítmica simple
        periodo, corr = detectar_ritmo(x_frame, fs)
        
        # Calcula un pulso visual que oscila con el ritmo detectado
        t_actual = i / FPS
        
        dibujar_circulo_ritmico(t_actual,periodo)

        # Visual: Círculo que sube/baja con pitch y cambia tamaño con envolvente
        dibujar_particula(pitch_frame[i], env_frame[i])
        
        dibujar_barras(X_resampled, N_BARRAS)

        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.axis('off')

        plt.tight_layout()
        plt.savefig(f"{FRAME_FOLDER}/frame_{i:04d}.png")
        plt.close(fig)
    
generar_frames(x,fs,FRAME_FOLDER)

out = "final.mp4"
crear_video(AUDIO_PATH,out)
Video("final.mp4")

Generando frames...
Completado 100.00 %
Procesado terminado.


### Resumen de PixelSounds: visualización de audio en video con Python

#### Objetivo general

Nuestro objetivo principal es el de crear un video dinámico a partir de un archivo de audio, donde la visualización reaccione a distintos descriptores del sonido como ritmo, espectro, envolvente, etc., utilizando exclusivamente Python.

#### Resumen de lo aprendido y conceptos explicados

##### Descriptores de audio
Hemos sabido comprender y aplicar diferentes descriptores que capturan distintas propiedades del sonido:

1. Envolvente de amplitud

* Representa la variación de la energía de la señal a lo largo del tiempo.

* Útil para detectar intensidad, ataques o silencios.

2. Espectro de frecuencias

* Se obtiene mediante la Transformada de Fourier.

* Permite ver qué frecuencias están presentes en un instante.

3. Centroide espectral

* Indica el “centro de gravedad” del espectro.

* Cuanto más alto, más brillante o agudo se percibe el sonido.

4. Autocorrelación

* Herramienta para detectar periodicidad o repetición.

* Aplicada sobre la envolvente para estimar el ritmo o tempo.

5. Detección de beats

* Se basa en picos periódicos de energía detectados en la envolvente.

* Permite sincronizar efectos visuales con los golpes musicales.

##### Procesamiento por fotogramas
* El audio se divide en ventanas temporales por fotogramas de vídeo.

* Cada fragmento se analiza individualmente para obtener descriptores y generar visuales sincronizados.

##### Relación audio → imagen
Hemos aprendido a convertir propiedades del audio en parámetros visuales:

* Pitch o frecuencia → posición vertical.

* Envolvente → tamaño o brillo.

* Centroide espectral → color.

* Beat → efectos puntuales (flashes, cambios bruscos).

Hemos aplicado conceptos de visualización dinámica, donde el vídeo no es estático, sino que evoluciona en función del sonido.

##### Programación y diseño de sistema
* Hemos optado por un diseño modular del sistema: funciones pequeñas y reutilizables para análisis, extracción de descriptores y visualización.

* Hemos hecho uso de librerías como matplotlib, scipy, numpy, colorsys.

* Hemos realizado la automatización de video + audio con ffmpeg.

#### Funciones creadas

* `normalizar(v)`: normaliza valores en rango [0, 1].

* `autocorrelacion(x_frame)`: autocorrelación normalizada de un fragmento.

* `detectar_ritmo(x_frame, fs, ...)`: calcula el periodo dominante.

* `es_beat(frame_index, beat_frames, ...)`: determina si un frame es un beat.

* `dibujar_flash()`: dibuja un flash central.

* `dibujar_barras(ax, X_resampled, N_BARRAS)`: visualización espectral.

* `color_fondo_por_centroide(x_frame, fs)`: color de fondo basado en centroide.

* `beat_frames(x, fs)`: calcula los beats a partir de la envolvente.
